# UCI-HAR

* Vision Blocks

In [1]:
block_list = ['ConvBlock', 'SeparableConvBlock', 'MBConvBlock', 'ResConvBlock', 'ShuffleBlock']

In [2]:
for block in block_list:
    for i in [1,3,5,7,9]:
        ! python convert_models/convert_visblock_mobile.py --dataset uci --block_name $block --kernel_size $i --skip_op None
        ! python convert_models/convert_visblock_mobile.py --dataset uci --block_name $block --kernel_size $i --skip_op identity

Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k1-None.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k1-identity.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k3-None.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k3-identity.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k5-None.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k5-identity.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k7-None.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k7-identity.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci/ConvBlock-k9-None.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt

* HAR Blocks

In [9]:
har_block_list = ["LSTMBlock", "BiLSTMBlock"] #, "GTSResConvBlock"]
har_lstmblock_list = ["LSTMBlock", "BiLSTMBlock"]

In [14]:
for block in har_block_list:
    ! python convert_models/convert_harblock_mobile.py --dataset uci --block_name $block
# for block in har_lstmblock_list:
#     ! python convert_models/convert_harblock_mobile.py --dataset uci --block_name $block --out_channels 128

Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci\LSTMBlock-k0.pt
Experiment dir : mobile_blocks_pt/uci
CPU model saved at : mobile_blocks_pt/uci\BiLSTMBlock-k0.pt


# OPP

* HAR model

In [3]:
har_model_list = ["RTWCNN", "HARLSTM", "HARBiLSTM", "HARConvLSTM", "ResNetTSC", "FCNTSC"] # TODO: "RTCNN"

In [9]:
for model in har_model_list:
    ! python convert_har_mobile.py --dataset opp --arch $model

Experiment dir : mobile_pt/opp
CPU model saved at : mobile_pt/opp\RTWCNN.pt
Experiment dir : mobile_pt/opp
CPU model saved at : mobile_pt/opp\HARLSTM.pt
Experiment dir : mobile_pt/opp
CPU model saved at : mobile_pt/opp\HARBiLSTM.pt
Experiment dir : mobile_pt/opp
CPU model saved at : mobile_pt/opp\HARConvLSTM.pt
Experiment dir : mobile_pt/opp
CPU model saved at : mobile_pt/opp\ResNetTSC.pt
Experiment dir : mobile_pt/opp
CPU model saved at : mobile_pt/opp\FCNTSC.pt


In [24]:
import torch
import torch.nn as nn

import rl_harnas.genotypes as genotypes
from rl_harnas.model_generator import BNConvReLU

genotype = eval("genotypes.{}".format('RLNAS'))

In [21]:
# # index / Type / Kernel Size / Pred1 / Pred2 / N_Kernels
# for g in genotype:
#     k = g[2]
#     nf = g[5]

# nf / kernel_size / padding
rlnas = [(g[5], g[2], int(g[2]/2)) for g in genotype[1:-1]]
rlnas

[(128, 1, 0),
 (32, 8, 4),
 (256, 5, 2),
 (256, 8, 4),
 (256, 8, 4),
 (256, 8, 4),
 (0, 0, 0),
 (256, 3, 1)]

In [26]:
nn.ModuleList([BNConvReLU(256, 256, 8, 4),BNConvReLU(256, 256, 8, 4)])

ModuleList(
  (0): BNConvReLU(
    (conv): Conv1d(256, 256, kernel_size=(8,), stride=(1,), padding=(4,))
    (ReLU): ReLU()
    (BN): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): BNConvReLU(
    (conv): Conv1d(256, 256, kernel_size=(8,), stride=(1,), padding=(4,))
    (ReLU): ReLU()
    (BN): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [27]:
32 + 128 + 256 + 256*3

1184

In [42]:

class RLNAS(nn.Module):
    def __init__(self, C, num_classes):
        super(RLNAS, self).__init__()
        self._C = C
        self._num_classes = num_classes
        self.get_network()
        
        self.lstm = nn.LSTM(1184, 128, num_layers=2, batch_first=True) # 32 + 128 + 256 + 256*3
        self.classifier = nn.Linear(128, self._num_classes)

    def get_network(self):
        # make each layer            
        self.layer1, self.layer2 = BNConvReLU(self._C, 128, 1, 0), BNConvReLU(self._C, 32, 8, 4)
        self.layer3 = BNConvReLU(128, 256, 5, 2)
        self.layer4, self.layer5, self.layer6 =  \
            BNConvReLU(256, 256, 8, 4), BNConvReLU(256, 256, 8, 4), BNConvReLU(256, 256, 8, 4)
        self.layer8 = BNConvReLU(256, 256, 3, 1)   
        # make layers
        self._layers = nn.ModuleList([
            self.layer1, self.layer2, 
            self.layer3, 
            self.layer4, self.layer5, self.layer6, 
            self.layer8
        ])       
        
    def forward(self, x):
        # input x -> x1, x2
        x1, x2 = self.layer1(x), self.layer2(x)
        # input x1 -> x3
        x3 = self.layer3(x1)
        # input x3 -> x4, x5, x6
        x4, x5, x6 = self.layer4(x3), self.layer5(x3), self.layer6(x3)
        x7, x8 = torch.cat([x1, x4], axis=1), self.layer8(x4)    
        
        # output
        x = torch.cat([x2, x5, x6, x7, x8], axis=1)
        
        # x = x.view(x.size(0), -1, 1184)
        # x, _ = self.lstm(x, h)
        # x = x.view(x.size(0), -1, 128)[:, -1, :]

        # x = self.classifier(x)

        # return x, h
    
        x = x.squeeze(-1).permute([0, 2, 1])
        lstm_out, (hn, cn) = self.lstm(x)
        last_time_step  = lstm_out[:, -1, :]
        logits = self.classifier(last_time_step)
        return logits
        

        

In [43]:
net = RLNAS(C=6, num_classes=6)

In [44]:
input = torch.randn(1, 6, 128)
net(input)

RuntimeError: Sizes of tensors must match except in dimension 1. Got 128 and 129 in dimension 2 (The offending index is 1)

# KUHAR

In [5]:
har_model_list = ["RTWCNN", "HARLSTM", "HARBiLSTM", "HARConvLSTM", "ResNetTSC", "FCNTSC"] # TODO: "RTCNN"

In [6]:
for model in har_model_list:
    ! python convert_har_mobile.py --dataset kar --arch $model

Experiment dir : mobile_pt/kar
CPU model saved at : mobile_pt/kar\RTWCNN.pt
Experiment dir : mobile_pt/kar
CPU model saved at : mobile_pt/kar\HARLSTM.pt
Experiment dir : mobile_pt/kar
CPU model saved at : mobile_pt/kar\HARBiLSTM.pt
Experiment dir : mobile_pt/kar
CPU model saved at : mobile_pt/kar\HARConvLSTM.pt
Experiment dir : mobile_pt/kar
CPU model saved at : mobile_pt/kar\ResNetTSC.pt
Experiment dir : mobile_pt/kar
CPU model saved at : mobile_pt/kar\FCNTSC.pt


# UniMib

In [ ]:
har_model_list = ["RTWCNN", "HARLSTM", "HARBiLSTM", "HARConvLSTM", "ResNetTSC", "FCNTSC"] # TODO: "RTCNN"

In [7]:
for model in har_model_list:
    ! python convert_har_mobile.py --dataset uni --arch $model

Experiment dir : mobile_pt/uni
CPU model saved at : mobile_pt/uni\RTWCNN.pt
Experiment dir : mobile_pt/uni
CPU model saved at : mobile_pt/uni\HARLSTM.pt
Experiment dir : mobile_pt/uni
CPU model saved at : mobile_pt/uni\HARBiLSTM.pt
Experiment dir : mobile_pt/uni
CPU model saved at : mobile_pt/uni\HARConvLSTM.pt
Experiment dir : mobile_pt/uni
CPU model saved at : mobile_pt/uni\ResNetTSC.pt
Experiment dir : mobile_pt/uni
CPU model saved at : mobile_pt/uni\FCNTSC.pt


# WISDM

In [ ]:
har_model_list = ["RTWCNN", "HARLSTM", "HARBiLSTM", "HARConvLSTM", "ResNetTSC", "FCNTSC"] # TODO: "RTCNN"

In [8]:
for model in har_model_list:
    ! python convert_har_mobile.py --dataset wis --arch $model

Experiment dir : mobile_pt/wis
CPU model saved at : mobile_pt/wis\RTWCNN.pt
Experiment dir : mobile_pt/wis
CPU model saved at : mobile_pt/wis\HARLSTM.pt
Experiment dir : mobile_pt/wis
CPU model saved at : mobile_pt/wis\HARBiLSTM.pt
Experiment dir : mobile_pt/wis
CPU model saved at : mobile_pt/wis\HARConvLSTM.pt
Experiment dir : mobile_pt/wis
CPU model saved at : mobile_pt/wis\ResNetTSC.pt
Experiment dir : mobile_pt/wis
CPU model saved at : mobile_pt/wis\FCNTSC.pt
